## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-17-04-48 +0000,nypost,Coldplay kiss cam’s Kristin Cabot soaks up the...,https://nypost.com/2026/02/16/us-news/coldplay...
1,2026-02-16-17-03-18 +0000,nypost,Hochul funnels $1.5B to NYC in partial bailout...,https://nypost.com/2026/02/16/us-news/hochul-f...
2,2026-02-16-17-01-42 +0000,nypost,Shocking moment gang of thieves loot Macy’s st...,https://nypost.com/2026/02/16/us-news/californ...
3,2026-02-16-17-00-00 +0000,wsj,The Newest Old Tech in Warfare: Balloons,https://www.wsj.com/politics/national-security...
4,2026-02-16-16-58-18 +0000,nyt,"Ticket Scam Cost the Louvre $12 Million, Inves...",https://www.nytimes.com/2026/02/16/arts/louvre...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2536/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,20
234,new,9
6,up,8
58,iran,6
147,guthrie,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,58
145,2026-02-16-05-39-01 +0000,bbc,Trump's new world order has become real and Eu...,https://www.bbc.com/news/articles/cddn002g6qzo...,45
16,2026-02-16-16-28-07 +0000,nypost,Trump threatens kidnappers with death penalty ...,https://nypost.com/2026/02/16/us-news/trump-te...,43
6,2026-02-16-16-57-55 +0000,nyt,Iran Holds Exercises in Strait of Hormuz After...,https://www.nytimes.com/2026/02/16/world/middl...,42
106,2026-02-16-11-00-00 +0000,nypost,Trump opposes AI safety bill in Utah after adm...,https://nypost.com/2026/02/16/us-news/trump-op...,37


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,58,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
72,34,2026-02-16-13-44-07 +0000,cbc,"Dana Eden, co-creator of Israeli TV series Teh...",https://www.cbc.ca/news/world/dana-eden-israel...
5,33,2026-02-16-16-57-58 +0000,cbc,"Obama clarifies comments on aliens being real,...",https://www.cbc.ca/news/world/obama-aliens-rea...
78,32,2026-02-16-12-36-03 +0000,bbc,'It's never too late': Savannah Guthrie's new ...,https://www.bbc.com/news/articles/c5yrv2g16e2o...
197,27,2026-02-15-19-54-32 +0000,cbc,'It was a gift': French goalie Junca's birthda...,https://www.cbc.ca/sports/olympics/winter/hock...
26,25,2026-02-16-16-05-58 +0000,bbc,Russian 'pick-up artist' accused of secretly f...,https://www.bbc.com/news/articles/c9wn5p299eko...
83,25,2026-02-16-12-28-45 +0000,nypost,Mom fatally shoots daughter before taking her ...,https://nypost.com/2026/02/16/us-news/mom-fata...
55,24,2026-02-16-14-55-55 +0000,nypost,Longtime NPR host accuses Google of stealing h...,https://nypost.com/2026/02/16/media/npr-host-d...
193,23,2026-02-15-20-15-29 +0000,nypost,Ex-Prince Andrew ghosting Oversight Committee’...,https://nypost.com/2026/02/15/us-news/ex-princ...
99,23,2026-02-16-11-09-22 +0000,nypost,NYC Health Department staffer wished Israelis ...,https://nypost.com/2026/02/16/us-news/nyc-heal...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
